In [1]:
from langchain.embeddings import HuggingFaceEmbeddings

# Sử dụng mô hình BERT (SBERT) để tạo embedding
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Tạo embedding từ văn bản
text = "LangChain là một framework mạnh mẽ cho AI."
vector = embedding_model.embed_query(text)

print(vector[:5])  # Hiển thị 5 giá trị đầu của vector embedding

[-0.07030469179153442, 0.0006977436714805663, 0.03368493914604187, -0.07487841695547104, -0.026996184140443802]


In [2]:
import pandas as pd
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv, find_dotenv
import pinecone
from sentence_transformers import SentenceTransformer

In [3]:
%load_ext dotenv
%dotenv

In [4]:
load_dotenv(find_dotenv(), override= True)

True

In [5]:
# Khởi tạo Pinecone
pc = Pinecone(api_key= os.environ.get("PINECONE_API_KEY"), environment = os.environ.get("PINECONE_ENV"))

# Tạo index nếu chưa có
index_name = "langchain-bert"
dimension = 384
metric = "cosine"

if index_name in [index.name for index in pc.list_indexes()]:
    pc.delete_index(index_name)
    print(f"{index_name} successfully delete")
else :
    print(f"{index_name} not in index list")


langchain-bert not in index list


In [8]:
pc.delete_index("my-index-2")

In [9]:
pc.create_index(
    name= index_name,
    dimension= dimension,
    metric= metric,
    spec= ServerlessSpec(
        cloud= "aws",
        region= "us-east-1"
    )
)

{
    "name": "langchain-bert",
    "metric": "cosine",
    "host": "langchain-bert-03idc8k.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [11]:
# Kết nối đến index
index = pc.Index(index_name)

In [12]:
# Thêm dữ liệu vào Pinecone
vector_id = "doc_1"
index.upsert(vectors=[(vector_id, vector)])

{'upserted_count': 1}

In [13]:
query_vector = embedding_model.embed_query("Công cụ AI mạnh mẽ")

# Thực hiện truy vấn trong Pinecone
results = index.query(vector=query_vector, top_k=3, include_metadata=True)

print("Top kết quả:", results)

Top kết quả: {'matches': [{'id': 'doc_1', 'score': 0.537551463, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}


In [14]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

In [16]:
# Tải mô hình T5 để tóm tắt văn bản
t5_pipeline = pipeline("summarization", model="t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


In [17]:
# Tích hợp vào LangChain
t5_model = HuggingFacePipeline(pipeline=t5_pipeline)

In [18]:
# Thử nghiệm với văn bản dài
long_text = """
LangChain là một framework hỗ trợ xây dựng ứng dụng AI dựa trên mô hình ngôn ngữ lớn (LLM).
Nó giúp kết nối LLM với cơ sở dữ liệu, API, và nhiều công cụ khác để xây dựng chatbot, hệ thống hỏi đáp, và tìm kiếm thông minh.
"""

In [19]:
summary = t5_model(long_text)
print("Tóm tắt:", summary)

Your max_length is set to 200, but your input_length is only 174. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=87)


Tóm tắt: LangChain là mt framework h tr xây dng ng AI . nó gip kt ni LLM và nhiu công c khác .


In [20]:
qa_pipeline = pipeline("text2text-generation", model="t5-small")

Device set to use cpu


In [21]:
# Tích hợp vào LangChain
qa_model = HuggingFacePipeline(pipeline=qa_pipeline)

In [22]:
question = "LangChain dùng để làm gì?"
answer = qa_model(question)
print("Câu trả lời:", answer)

Câu trả lời: LangChain dùng  làm g?


In [23]:
# Tạo vector database từ Pinecone
vectorstore = Pinecone(embedding_model, index_name=index_name)

In [26]:
from langchain.chains import RetrievalQA